In [0]:
!pip3 install --quiet "tensorflow>=1.7"
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [2]:
!rm edAR_text_similarity
!git clone https://github.com/lkoshale/edAR_text_similarity.git

rm: cannot remove 'edAR_text_similarity': No such file or directory
Cloning into 'edAR_text_similarity'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 48 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [3]:
# importing libraries
import tensorflow as tf
import tensorflow_hub as hub
import re
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from keras import layers

Using TensorFlow backend.


In [0]:
# google universal encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [0]:
#  Preprocessing File
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(":")[0]
        text = ' '.join(lines[i].split(' ')[1:])
        text = re.sub('[^A-Za-z0-9 ,\?\'\"-._\+\!/\`@=;:]+', '', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

In [7]:
df_train = get_dataframe('edAR_text_similarity/data/train_5500.txt')
df_test = get_dataframe('edAR_text_similarity/data/test_500.txt')
df_test.head()

,label,text
0,NUM,How far is it from Denver to Aspen ?
1,LOC,"What county is Modesto , California in ?"
2,HUM,Who was Galileo ?
3,DESC,What is an atom ?
4,NUM,When did Hawaii become a state ?


In [9]:
df_test.label

0       NUM
1       LOC
2       HUM
3      DESC
4       NUM
       ... 
495     HUM
496    ENTY
497     NUM
498    ENTY
499    DESC
Name: label, Length: 500, dtype: category
Categories (6, object): [ABBR, DESC, ENTY, HUM, LOC, NUM]

In [0]:
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

test_label = np.asarray(pd.get_dummies(df_test['label']), dtype = np.int8)



In [11]:
test_label


array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=int8)

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [13]:
# building model
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(512,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(6, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 132,870
Trainable params: 132,870
Non-trainable params: 0
_________________________________________________________________


In [14]:
# training model
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,
            batch_size=32)
  model.save_weights('./model.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5452 samples, validate on 500 samples
Epoch 1/10



5452/5452 [==============================] - 66s 12ms/step - loss: 0.7217 - acc: 0.8098 - val_loss: 0.3157 - val_acc: 0.9000
Epoch 2/10
5452/5452 [==============================] - 64s 12ms/step - loss: 0.3244 - acc: 0.8947 - val_loss: 0.2424 - val_acc: 0.9260
Epoch 3/10
5452/5452 [==============================] - 64s 12ms/step - loss: 0.2818 - acc: 0.9052 - val_loss: 0.2237 - val_acc: 0.9240
Epoch 4/10
5452/5452 [==============================] - 63s 12ms/step - loss: 0.2613 - acc: 0.9087 - val_loss: 0.2262 - val_acc: 0.9340
Epoch 5/10
5452/5452 [==============================] - 64s 12ms/step - loss: 0.2498 - acc: 0.9167 - val_loss: 0.2313 - val_acc: 0.9260
Epoch 6/10
5452/5452 [==============================] - 64s 12ms/step - loss: 0.2362 - acc: 0.9202 - val_loss: 0.2048 - val_acc: 0.9360
Epoch 7/10
5452/5452 [==============================] - 64s 12ms/step - loss: 0.2231 - acc: 0.9233 - val_loss: 0.2031 - val_acc: 0.9380
Epoch 8/10
5452/5452 [==============================] - 64s

In [0]:
#testing model

new_text = ["In what year did the titanic sink ?", "What is the highest peak in California ?", "Who invented the light bulb ?"]
new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text, batch_size=32)

In [16]:
predicts

array([[6.3573241e-10, 6.5678942e-05, 9.2486232e-05, 2.4987779e-07,
        9.9699164e-06, 9.9983156e-01],
       [1.3267380e-07, 1.4936492e-03, 8.6584809e-04, 4.7873866e-05,
        5.8158547e-01, 4.1600701e-01],
       [7.4548305e-09, 1.3597630e-06, 1.0655369e-04, 9.9988699e-01,
        4.7854028e-06, 3.1105475e-07]], dtype=float32)

In [17]:
categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
predict_labels

['NUM', 'LOC', 'HUM']